# Chroma DB

In [2]:
import chromadb
chroma_client = chromadb.Client()


In [9]:
collection = chroma_client.create_collection(name="new_collection")


# Generate a embedding

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [8]:
emb1 = embeddings.embed_query("This is a document about pineapple")
emb2 = embeddings.embed_query("This is a document about oranges")

In [10]:
collection.add(
    embeddings=[emb1, emb2],
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)


In [12]:
emb_query = embeddings.embed_query("This is a query document about hawaii")

results = collection.query(
    query_embeddings=emb_query,
    query_texts=["This is a query document about hawaii"],
    n_results=2
)
print(results)


{'ids': [['id1', 'id2']], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None, 'metadatas': [[None, None]], 'distances': [[0.2704945206642151, 0.3485858142375946]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


# Let's persist the DB

In [34]:
import chromadb

client = chromadb.PersistentClient(path="chroma")


In [35]:
collection = chroma_client.get_or_create_collection(name="facts-2")


In [41]:
collection.count()

10

In [18]:
statements = [
    "The sun rises in the east.", "The sun appears from the eastern horizon in the morning.",
    "Water boils at 100°C at sea level.", "At sea level, water reaches its boiling point at 100 degrees Celsius.",
    "Dogs are loyal animals.", "Canines are known for their loyalty.",
    "Paris is the capital of France.", "The capital city of France is Paris.",
    "Exercising regularly improves health.", "Regular physical activity benefits overall well-being."
]

In [20]:
metadatas = [{"len": len(s)} for s in statements]

In [19]:
embds = embeddings.embed_documents(texts=statements)

In [37]:
import uuid

collection.upsert(
    embeddings=embds,
    documents=statements,
    ids=[str(uuid.uuid4()) for _ in range(len(statements))],
    metadatas=metadatas,
)

In [38]:
emb_query = embeddings.embed_query("countries")

results = collection.query(
    query_embeddings=emb_query,
    query_texts=["country"],
    n_results=2
)
print(results)

{'ids': [['4f9e3a8e-8b62-41bb-983b-0ee357dcf644', '494847b9-103d-45df-b02e-ba84f0208019']], 'embeddings': None, 'documents': [['The capital city of France is Paris.', 'Paris is the capital of France.']], 'uris': None, 'data': None, 'metadatas': [[{'len': 36}, {'len': 31}]], 'distances': [[0.4284917712211609, 0.4303634762763977]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [39]:
filter = {"$and": [{"len": {"$gte": 32}}, {"len": {"$lte": 40}}]}

In [40]:
results = collection.query(
    query_embeddings=emb_query,
    query_texts=["country"],
    n_results=1,
    where=filter,
)
print(results)

{'ids': [['4f9e3a8e-8b62-41bb-983b-0ee357dcf644']], 'embeddings': None, 'documents': [['The capital city of France is Paris.']], 'uris': None, 'data': None, 'metadatas': [[{'len': 36}]], 'distances': [[0.4284917712211609]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


# Let's use langchain

In [42]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [45]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [46]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['9b215376-1223-4b6c-9cd2-dadd0cef2654',
 'cfa35a55-af14-400f-a92a-49d16c240e00',
 '31586b81-f34a-499d-95f3-2c0bc7dd18d0',
 '3e7cf343-0d6b-46db-a296-e2d207dfd900',
 '9a045f53-8634-4d45-b598-cc26794e2ebb',
 'd847c44d-ca2c-4dee-8b28-7c991953f8e9',
 '35565c6a-4ff0-4102-bb99-95ad9fbd2296',
 '42acb79e-40f7-4307-b06f-8c38db109a85',
 '878364c8-024d-4824-ac55-8a620eab5e5c',
 'e5a520f6-ca06-4331-9282-61ed7bac18d3']

In [47]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
